In [1]:
import pandas as pd
import re
import psycopg2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import schedule
import time

In [13]:
data = pd.read_excel("Train.xlsx")

In [14]:
stemmer = StemmerFactory().create_stemmer()
indonesian_stopwords = set(stopwords.words('indonesian'))

def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = stemmer.stem(text)
    text = ' '.join([word for word in text.split() if word not in indonesian_stopwords])
    return text

data['Case_normalized'] = data['Case'].apply(normalize_text)
data.head()


,Case,EDC,Complain Catagory,Urgency,EDC Range,Case_normalized
0,tolong bantu reset password dashboard saya,4,AGEN KNOWLEDGE,Low,2,tolong bantu reset password dashboard
1,bisa tolong periksa transaksi yang mengalami m...,3,AGEN KNOWLEDGE,Low,2,tolong periksa transaksi alami akun
2,bantu cek transaksi saya tanggal 19 Agustus,3,AGEN KNOWLEDGE,Low,2,bantu cek transaksi tanggal agustus
3,Kenapa transaksi saya selalu berakhir dengan e...,3,AGEN KNOWLEDGE,Low,2,transaksi error
4,Saya butuh bantuan untuk reset password dashbo...,3,AGEN KNOWLEDGE,Low,2,butuh bantu reset password dashboard


In [15]:
# Vektorisasi kolom 'case_normalized' menggunakan TF-IDF
vectorizer = TfidfVectorizer()
X_case = vectorizer.fit_transform(data['Case_normalized'])

In [16]:
# Encode kolom EDC
edc_encoder = LabelEncoder()
data['EDC_encoded'] = edc_encoder.fit_transform(data['EDC Range'])
# menggabungkan 'Case' dan 'EDC' sebagai fitur
X = pd.concat([pd.DataFrame(X_case.toarray()), data['EDC_encoded']], axis=1)
X.columns = X.columns.astype(str)

In [17]:
# Encode kolom target 'Complain Category' dan 'Urgency'
category_encoder = LabelEncoder()
y_category = category_encoder.fit_transform(data['Complain Catagory'])

urgency_encoder = LabelEncoder()
y_urgency = urgency_encoder.fit_transform(data['Urgency'])

In [33]:
X_train_category, X_test_category, y_train_category, y_test_category = train_test_split(X_case, y_category, test_size=0.2, random_state=45123)
X_train_urgency, X_test_urgency, y_train_urgency, y_test_urgency = train_test_split(X, y_urgency, test_size=0.2, random_state=51234)

In [34]:
# Train Multinomial Naive Bayes Model untuk Complain Category dan Urgency
model_category = MultinomialNB().fit(X_train_category, y_train_category)
model_urgency = MultinomialNB().fit(X_train_urgency, y_train_urgency)

# Predictions
y_pred_category = model_category.predict(X_test_category)
y_pred_urgency = model_urgency.predict(X_test_urgency)

In [35]:
print("Complain Category Classification Report:")
print(classification_report(y_test_category, y_pred_category))

print("Urgency Classification Report:")
print(classification_report(y_test_urgency, y_pred_urgency))

Complain Category Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.68      0.79        31
           1       0.95      0.72      0.82        29
           2       0.61      0.96      0.75        26
           3       0.97      0.97      0.97        30
           4       0.94      0.94      0.94        34
           5       1.00      1.00      1.00        24
           6       0.97      1.00      0.98        29

    accuracy                           0.89       203
   macro avg       0.91      0.90      0.89       203
weighted avg       0.92      0.89      0.89       203

Urgency Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98        73
           1       0.78      0.79      0.79        63
           2       0.78      0.76      0.77        67

    accuracy                           0.85       203
   macro avg       0.85      0.85      0.85       203
weigh

In [10]:
db_params = {
    'host': 'localhost',
    'dbname': 'Indopay_HRIS',
    'user': 'postgres',
    'password': '1Agustus',
    'port': 5432
}

# Category mapping for insertion
category_mapping = {
    "AGEN KNOWLEDGE": 1,
    "EDC HARDWARE": 2,
    "EDC SOFTWARE": 3,
    "GPRS SIGNAL": 4,
    "POWER SUPPLY": 5,
    "REQUEST PAPER": 6,
    "REQUEST PULL OUT": 7
}

# Urgency mapping for insertion
urgency_mapping = {
    "Low": 1,
    "Normal": 2,
    "High": 3
}



In [ ]:
def check_and_update_tickets():
    print("Starting ticket update process")
    try:
        conn = psycopg2.connect(**db_params)
        cur = conn.cursor()

        query_case = '''
        SELECT 
            t."TID", 
            (t.title || ' ' || t.note) AS case, 
            m."EDC" 
        FROM 
            tiket t
        JOIN 
            merchant m ON t."MID" = m."MID"
        WHERE 
            t.urgency_id IS NULL
            OR t.category_id IS NULL;
        '''

        cur.execute(query_case)
        results = cur.fetchall()

        for result in results:
            tid = result[0]
            new_case = result[1]
            edc_value = result[2]

            new_case_normalized = normalize_text(new_case)
            X_new_case = vectorizer.transform([new_case_normalized])
            edc_encoded = edc_encoder.transform([edc_value])

            # menggabungkan case dengan jumlah EDC untuk klasifikasi urgensi
            X_new_combined = pd.concat([pd.DataFrame(X_new_case.toarray()), pd.DataFrame(edc_encoded, columns=['EDC_encoded'])], axis=1)
            X_new_combined.columns = X.columns.astype(str)

            # mengklasifikasi tiket
            predicted_category = model_category.predict(X_new_case)
            predicted_urgency = model_urgency.predict(X_new_combined)

            # Decode hasil prediksi
            complain_category_label = category_encoder.inverse_transform(predicted_category)[0]
            urgency_label = urgency_encoder.inverse_transform(predicted_urgency)[0]

            # Insert ke database berdasarkan TID
            if complain_category_label in category_mapping and urgency_label in urgency_mapping:
                category_id = category_mapping[complain_category_label]
                urgency_id = urgency_mapping[urgency_label]

                insert_query = '''
                UPDATE tiket 
                SET "category_id" = %s, "urgency_id" = %s 
                WHERE "TID" = %s;
                '''
                cur.execute(insert_query, (category_id, urgency_id, tid))
                conn.commit()
                print(f"Inserted category_id {category_id} and urgency_id {urgency_label} for TID {tid}")
            else:
                print(f"Unknown complain category: {complain_category_label}")

    except Exception as error:
        print(error)

    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

# jalankan setiap 1 menit
schedule.every(1).minutes.do(check_and_update_tickets)

while True:
    schedule.run_pending()
    time.sleep(1)

Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Starting ticket update process
Inserted category_id 6 and urgency_id Low for TID ID081348
Starting ti